In [ ]:
# pip
# !pip install tqdm
# !pip install dask
# !pip install apache-sedona
# !pip install shapely
# !pip install geopandas
# !pip install leafmap
# !pip install openpyxl
# !pip install --upgrade ipyleaflet
# !jupyter labextension install jupyter-leaflet
# !jupyter labextension install @jupyter-widgets/jupyterlab-manager

In [ ]:
# conda
# !conda install tqdm
# !conda install dask
#!conda install findspark

In [1]:
from sedona.register import SedonaRegistrator  
from sedona.utils import SedonaKryoRegistrator, KryoSerializer
import geopandas as gpd
from shapely.geometry import Point, Polygon
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from haversine import haversine, Unit
import pyspark.sql.types as types
import matplotlib.pyplot as plt
import matplotlib
import matplotlib.dates as mdates
import leafmap

In [2]:
# configuration, worked on using python@3.10.9 
import os
import urllib
import json
from threading import Thread, Lock
from tqdm import tqdm
from kafka import KafkaConsumer, KafkaProducer
import pyspark
from pyspark.sql import SparkSession
import pandas

In [3]:
'''conf = pyspark.SparkConf()\
    .setMaster("local[7]")\
    .set("spark.eventLog.enabled", "true")\
    .set("spark.eventLog.dir", "./logs")\
    .set("spark.eventLog.gcMetrics.youngGenerationGarbageCollectors", "true")\
    .set("spark.executor.heartbeatInterval","3600s")\
    .set("spark.network.timeout","3601s")\
    .set("spark.sql.repl.eagerEval.enabled", False)



sc = pyspark.SparkContext(conf=conf)
sc.setLogLevel('ERROR')
spark = pyspark.sql.SparkSession(sc)
spark
'''

# spark session initialization
spark = SparkSession.builder\
    .master("local[5]")\
    .appName("main")\
    .config("spark.sql.debug.maxToStringFields", 50)\
    .config("spark.driver.memory", '8g')\
    .config("spark.executor.instances", 4)\
    .config("spark.executor.cores", 5)\
    .getOrCreate()

spark

In [4]:
spark.conf.get("spark.executor.cores")

'5'

In [5]:
# data folder
data_dir = 'data'

# data urls
# historic_arrest_loc = { 'url': 'https://data.cityofnewyork.us/resource/8h9b-rp9u.json?$limit=10', 'filename': 'lil_arrest.json' }
historic_arrest_loc = { 'url': 'https://data.cityofnewyork.us/resource/8h9b-rp9u.csv?$limit=15000000', 'filename': 'arrest.csv' }
historic_complaint_loc = { 'url': 'https://data.cityofnewyork.us/resource/qgea-i56i.json?$limit=15000000', 'filename': 'complaint.json' }
historic_court_summons_loc = { 'url': 'https://data.cityofnewyork.us/resource/sv2w-rv3k.json?$limit=10', 'filename': 'lil_summons.json' }
traffic_speed_loc = { 'url': 'https://data.cityofnewyork.us/resource/i4gi-tjb9.json?$limit=15000000', 'filename': 'speed.json' }
turnstile_loc = { 'url': 'https://data.ny.gov/resource/i55r-43gk.json?$limit=15000000', 'filename': 'turnstile.json' }
#subway_loc = { 'url': 'https://data.ny.gov/resource/i9wp-a4ja.json?$limit=15000000', 'filename': 'subway.json' }
subway_loc = { 'url': 'http://web.mta.info/developers/data/nyct/subway/Stations.csv?$limit=15000000', 'filename': 'subway.csv' }

In [6]:
# download flags
downloadflag = True
redownload = False

thread_lock = Lock()

# download utils
def download_dataset_thread(loc, folder):
    with thread_lock:
         if ((not os.path.exists(os.path.join(folder, loc['filename']))) or redownload) and downloadflag:
            if os.path.isfile(os.path.join(folder, loc['filename'])):
                os.remove(os.path.join(folder, loc['filename']))
            if not os.path.exists(folder):
                os.makedirs(folder) 
            with tqdm(unit="B", unit_scale=True, desc=loc['filename'], miniters=1) as progress_bar:
                urllib.request.urlretrieve(loc['url'], os.path.join(folder, loc['filename']), lambda block_num, block_size, total_size: progress_bar.update(block_size))
            progress_bar.display()
        
def download_dataset(loc, folder):
    thread = Thread(target=download_dataset_thread, args=(loc, folder))
    thread.start()
    thread.join()
        

In [7]:
# download datasets
for dataset in [historic_arrest_loc,
                historic_complaint_loc,
                historic_court_summons_loc,
                turnstile_loc,
                subway_loc]:
    download_dataset(dataset, data_dir)

In [ ]:
# dataframes
arrest_rdd = spark.read.json(os.path.join(data_dir, historic_arrest_loc['filename']), multiLine=True)
complaint_rdd = spark.read.json(os.path.join(data_dir, historic_complaint_loc['filename']), multiLine=True)
summons_rdd = spark.read.json(os.path.join(data_dir, historic_court_summons_loc['filename']), multiLine=True)
turnstile_df = spark.read.json(os.path.join(data_dir, turnstile_loc['filename']), multiLine=True).repartition(5)
subway_rdd = spark.read.json(os.path.join(data_dir, subway_loc['filename']), header=True, inferSchema=True)

In [ ]:
turnstile_df = spark.read.json(os.path.join(data_dir, turnstile_loc['filename']), multiLine=True).repartition(5)

In [8]:
#subway_rdd = spark.read.json(os.path.join(data_dir, subway_loc['filename']), multiLine=True)
#subway_rdd = spark.read.csv(os.path.join(data_dir, subway_loc['filename']), header=True, inferSchema=True)
# subway_df = spark.read.csv(os.path.join(data_dir, subway_loc['filename']), header=True, inferSchema=True).repartition(5)

In [12]:
subway_DF = subway_rdd.toDF("Station ID", "Complex ID", "GTFS Stop ID", "Division", "Line", "Stop Name", "Borough", "Daytime")

IllegalArgumentException: requirement failed: The number of columns doesn't match.
Old column names (19): Station ID, Complex ID, GTFS Stop ID, Division, Line, Stop Name, Borough, Daytime Routes, Structure, GTFS Latitude, GTFS Longitude, North Direction Label, South Direction Label, ADA, ADA Direction Notes, ADA NB, ADA SB, Capital Outage NB, Capital Outage SB
New column names (8): Station ID, Complex ID, GTFS Stop ID, Division, Line, Stop Name, Borough, Daytime

In [ ]:
#arrest_rdd = spark.read.json(os.path.join(data_dir, historic_arrest_loc['filename']), multiLine=True)

In [ ]:
file_location = "C:\\Users\\Nigel\\Github\\big-data-project\\data\\arrest.csv"
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
arrest_DF = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

display(arrest_DF)

In [ ]:
arrest_DF.count()

In [9]:
summons_rdd = spark.read.json(os.path.join(data_dir, historic_court_summons_loc['filename']), multiLine=True)

In [ ]:
lil_summons_rdd = spark.read.json(os.path.join(data_dir, historic_court_summons_loc['filename']), multiLine=True)

In [ ]:
lil_arrest_rdd = spark.read.json(os.path.join(data_dir, historic_arrest_loc['filename']), multiLine=True)

In [ ]:
# normalize udf for station id in subway and turnstile
def normalize(unit):
    type, *val = list(unit)
    return type + str(int(''.join(val)))

nUdf = F.udf(normalize, types.StringType())

In [10]:
# normalize subway GTFS Stop ID
subway_df = subway_df.withColumn('GTFS Stop ID', nUdf(F.col('GTFS Stop ID')))

NameError: name 'subway_df' is not defined

In [ ]:
# normalize turnstile unit
turnstile_df = turnstile_df.withColumn('unit', nUdf(F.col('unit')))

In [ ]:
# getting subways stations in turnstile and subway dataset
s_list = subway_df.select(F.col('GTFS Stop ID')).distinct().toPandas().values.flatten()
t_list = turnstile_df.select(F.col('unit')).distinct().toPandas().values.flatten()

In [ ]:
# comparing common subway stations
ts_intersect = [value for value in t_list if value in s_list]
ts_intersect

In [ ]:
gt_df = turnstile_df.groupBy(F.col('unit'))\
    .agg({ 'entries': 'sum', 'exits': 'sum'})\
        .select(F.col('unit'),\
            F.col('sum(entries)').alias('entries'),\
            F.col('sum(exits)').alias('exits'))
gt_df.show()

In [ ]:
# combine the total exits and entries for each station
st_df = subway_df.join(gt_df, F.col('GTFS Stop ID') == F.col('unit'))\
    .select(F.col('GTFS Stop ID').alias('id'),\
        F.col('Line').alias('line'),\
        F.col('Stop Name').alias('stop_name'),\
        F.col('Borough').alias('borough'),\
        F.col('GTFS Latitude').alias('lat'),\
        F.col('GTFS Longitude').alias('long'),\
        F.col('North Direction Label').alias('n_label'),\
        F.col('South Direction Label').alias('s_label'),\
        F.col('entries'),\
        F.col('exits'))
st_df.show()

In [ ]:
# subway_DF = subway_rdd.toDF("ada", "ada_notes", "corner", "division", "east_west_street", "entrance_georeference", "entrance_latitude", "entrance_location", "entrance_longitude", "entrance_type", "entry", "exit_only", "free_crossover", "line", "north_south_street", "route1", "route10", "route11", "route2", "route3", "route4", "route5", "route6", "route7", "route8", "route9", "staff_hours", "staffing", "station_georeference", "station_latitude", "station_location", "station_longitude", "station_name", "vending")
subway_DF = subway_rdd.toDF("Station ID", "Complex ID", "GTFS Stop ID", "Division", "Line", "Stop Name", "Borough", "Daytime Routes", "Structure", "GTFS Latitude", "GTFS Longitude", "North Direction Label", "South Direction Label", "ADA", "ADA Direction Notes", "ADA NB", "ADA SB", "Capital Outage NB", "Capital Outage SB")

In [ ]:
#subway_DF = subway_DF.select("station_name","station_latitude", "station_longitude", "station_location", "station_georeference")
subway_DF = subway_DF.select('Station ID', 'Complex ID', 'GTFS Stop ID', 'Stop Name', 'Borough', 'GTFS Latitude','GTFS Longitude')\
                     .filter(F.col('GTFS Latitude').isNotNull() & F.col('GTFS Longitude').isNotNull())

In [ ]:
subway_DF.show()

In [ ]:
subway_DF.printSchema()

In [ ]:
subway_DF.count()

In [ ]:
# CONSIDER DROPPING SUBWAY STOPS WITH DUPLICATE NAMES?
# subway_DF.dropDuplicates(subset=["Stop Name"]).count()

Adapting the Haversine function from Homework 2. Setting it to check distance in meters between lat/lon pairs

In [ ]:
def withinMeters(slat, slong, dlat, dlong):
    srs = (slat, slong)
    dst = (dlat, dlong)
    # print(type(srs[0]),type(srs[1]))
    # print(srs[0],srs[1])
    # print(type(dst[0]),type(dst[1]))
    # print(dst[0],dst[1])
    distance = float(haversine(srs, dst,unit=Unit.METERS))
    # print(distance)
    return bool(distance < 402)

    
withinMetersUdf = F.udf(withinMeters, types.BooleanType())

Tried to get the join to work on the full arrest data set but failed and have commented it out

In [ ]:
# arrest_DF = arrest_rdd.toDF(":@computed_region_92fq_4b7q", ":@computed_region_efsh_h5xi", ":@computed_region_f5dn_yrer", ":@computed_region_sbqj_enih", ":@computed_region_yeji_bk3q", "age_group", "arrest_boro", "arrest_date", "arrest_key", "arrest_precinct", "jurisdiction_code", "ky_cd", "latitude", "law_cat_cd", "law_code", "lon_lat", "longitude", "ofns_desc", "pd_cd", "pd_desc", "perp_race", "perp_sex", "x_coord_cd", "y_coord_cd")

In [ ]:
arrest_DF = arrest_DF.select("arrest_boro","arrest_date", "arrest_key", "latitude", "longitude", ).filter(F.col('latitude').isNotNull() & F.col('longitude').isNotNull()).withColumn("longitude", F.col("longitude").cast("double")).withColumn("latitude", F.col("latitude").cast("double"))

In [ ]:
subway_arrest_DF = subway_DF.crossJoin(arrest_DF)

In [ ]:
subway_arrest_DF.printSchema()

In [ ]:
subway_arrest_DF.count()

In [ ]:
subway_arrest_DF = subway_arrest_DF.withColumn("haversine", withinMetersUdf(F.col("GTFS Latitude"), F.col("GTFS Longitude"), F.col("latitude"), F.col("longitude")))

In [ ]:
subway_arrest_DF.count()

In [ ]:
subway_arrest_DF.printSchema()

In [ ]:
filtered_subway_arrest_DF = subway_arrest_DF.filter(F.col('haversine') == True)

In [ ]:
filtered_subway_arrest_DF.count()

In [ ]:
!java -version

In [ ]:
filtered_subway_arrest_DF = subway_arrest_DF.select('Stop Name', 'GTFS Latitude', 'GTFS Longitude',\
    'Borough', 'haversine')\
    .where(subway_arrest_DF.haversine == 1)

In [ ]:
filtered_subway_arrest_DF.groupBy(F.col('Stop Name')).agg(F.count("*").alias("Count"))

In [ ]:
filtered_subway_arrest_DF.count()

In [ ]:
# subway_arrest_DF = subway_DF.join(arrest_DF, withinMetersUdf('GTFS Latitude', 'GTFS Longitude', 'latitude', 'longitude'), 'cross')\
    # .drop(F.col('latitude'))\
    # .drop(F.col('longitude'))

In [ ]:
# subway_DF.printSchema()

In [ ]:
# arrest_count = subway_arrest_DF.groupBy(F.col('Stop Name')).count()
# arrest_count = arrest_count.dropDuplicates(subset=["Stop Name"])

In [ ]:
# arrest_count.printSchema()

In [ ]:
# arrest_count.count()

Building the smaller arrest dataset (10 rows) and trying to join with the subway data using the haversine function.

In [ ]:
lil_arrest_DF = lil_arrest_rdd.toDF(":@computed_region_92fq_4b7q", ":@computed_region_efsh_h5xi", ":@computed_region_f5dn_yrer", ":@computed_region_sbqj_enih", ":@computed_region_yeji_bk3q", "age_group", "arrest_boro", "arrest_date", "arrest_key", "arrest_precinct", "jurisdiction_code", "ky_cd", "latitude", "law_cat_cd", "law_code", "lon_lat", "longitude", "ofns_desc", "pd_cd", "pd_desc", "perp_race", "perp_sex", "x_coord_cd", "y_coord_cd")

In [ ]:
lil_arrest_DF = lil_arrest_DF.select("arrest_boro","arrest_date", "arrest_key", "latitude", "longitude", ).filter(F.col('latitude').isNotNull() & F.col('longitude').isNotNull()).withColumn("longitude", F.col("longitude").cast("double")).withColumn("latitude", F.col("latitude").cast("double"))

In [ ]:
lil_arrest_DF.show()

In [ ]:
lil_arrest_DF.printSchema()

Joining the subway dataframe to the smaller arrest data set unsuccessfully. I tried this with just the strings for the column names and with the F.col() structure but didn't have any success either way

In [ ]:
subway_arrest_DF = subway_DF.join(lil_arrest_DF, withinMetersUdf(F.col('GTFS Latitude'), F.col('GTFS Longitude'), F.col('latitude'), F.col('longitude')), 'cross')\
    .drop(F.col('latitude'))\
    .drop(F.col('longitude'))

In [ ]:
# Compute haversine after doing a general crossjoin and create a new column with T/F
# Then filter on new column (T/F)

The error occurs when trying to execute the pipeline graph with the show() below

In [ ]:
subway_arrest_DF = subway_arrest_DF.na.drop(subset=["Stop Name"])
lil_arrest_count = subway_arrest_DF.groupBy(F.col('Stop Name')).count()
lil_arrest_count.printSchema()
lil_arrest_count.show()

Testing the haversine function locally and it works so it seems like there's an issue passing the columns? I checked the hw2 submission and it seems fine

In [ ]:
distance = withinMeters(40.799008797000056, -73.95240854099995, 40.816391847000034, -73.89529641399997)
distance

In [ ]:
summons_DF = summons_rdd.toDF(":@computed_region_92fq_4b7q", ":@computed_region_efsh_h5xi", ":@computed_region_f5dn_yrer", ":@computed_region_sbqj_enih", ":@computed_region_yeji_bk3q", "age_group", "boro", "geocoded_column", "jurisdiction_code", "latitude", "law_description", "law_section_number", "longitude", "offense_description", "precinct_of_occur", "race", "sex", "summons_category_type", "summons_date", "summons_key", "x_coordinate_cd", "y_coordinate_cd")\
                        .select("boro","latitude", "longitude","offense_description","summons_category_type", "summons_date", "summons_key").filter(F.col('latitude').isNotNull() & F.col('longitude').isNotNull()).withColumn("longitude", F.col("longitude").cast("double")).withColumn("latitude", F.col("latitude").cast("double"))

In [ ]:
summons_DF.printSchema()

In [ ]:
lil_summons_DF = lil_summons_rdd.toDF(":@computed_region_92fq_4b7q", ":@computed_region_efsh_h5xi", ":@computed_region_f5dn_yrer", ":@computed_region_sbqj_enih", ":@computed_region_yeji_bk3q", "age_group", "boro", "geocoded_column", "jurisdiction_code", "latitude", "law_description", "law_section_number", "longitude", "offense_description", "precinct_of_occur", "race", "sex", "summons_category_type", "summons_date", "summons_key", "x_coordinate_cd", "y_coordinate_cd")\
                        .select("boro","latitude", "longitude","offense_description","summons_category_type", "summons_date", "summons_key").filter(F.col('latitude').isNotNull() & F.col('longitude').isNotNull()).withColumn("longitude", F.col("longitude").cast("double")).withColumn("latitude", F.col("latitude").cast("double"))

In [ ]:
lil_summons_DF.show(truncate = True)

In [ ]:
lil_summons_DF.printSchema()

Figuring out the map plotting for the subway locations in geopandas

In [ ]:
subway_DF.show()

In [14]:
test_DF = subway_DF.withColumn("arrests", ((F.rand() * 80000) + 20000).cast("integer"))

NameError: name 'subway_DF' is not defined

In [ ]:
test_DF.show()

Once we have the dataset bounded at <500 rows (a row per station), we should convert to pandas

In [ ]:
test_DF = test_DF.toPandas()

In [ ]:
test_DF.head(10)

In [ ]:
test_DF['geometry'] = [Point(xy) for xy in zip(test_DF['GTFS Longitude'],test_DF['GTFS Latitude'])]

In [ ]:
test_DF["arrest_pct_total"] = (test_DF["arrests"] / test_DF["arrests"].sum()) * 100

In [ ]:
test_DF = test_DF.sort_values(by='arrest_pct_total', ascending=False)

In [ ]:
test_DF.head()

In [ ]:
test_DF["Borough"].unique()

In [ ]:
nta_map = gpd.read_file(r'C:\Users\Nigel\Github\big-data-project\data\nynta2020_23a\nynta2020.shp')
nta_map.to_crs(4326, inplace=True)

In [ ]:
test_geo_DF = gpd.GeoDataFrame(test_DF, crs=4326, geometry = test_DF.geometry)
# Just to be extra sure
test_geo_DF.to_crs(4326, inplace=True)

In [ ]:
fig,ax = plt.subplots(figsize=(7,7))
nta_map.boundary.plot(ax=ax, edgecolor='k');
test_geo_DF[test_geo_DF.Borough == 'M']\
            .head(10)\
            .plot(column='Stop Name', ax=ax, legend=True, marker='.',\
            markersize=test_geo_DF.arrest_pct_total.astype('float') * 10000);

In [ ]:
lil_test_geo_DF = test_geo_DF.head(10)

In [ ]:
nta_demos = pandas.read_excel('https://www1.nyc.gov/assets/planning/download/office/planning-level/nyc-population/acs/demo_2019_acs5yr_nta.xlsx')

In [ ]:
nta_df = nta_map.merge(nta_demos, how='left', left_on='NTA2020', right_on='GeoID')

In [ ]:
lil_test_geo_DF.to_file("test_geo.json", driver="GeoJSON")

In [ ]:
m = leafmap.Map(center=(40,-100),zoom=4)
m.add_gdf(nta_df, layer_name='2020 NTA Demographic Information', info_mode='on_click')
m.add_point_layer(filename=r'C:\Users\Nigel\Github\big-data-project\test_geo.json', popup=['Stop Name', 'arrests', 'arrest_pct_total'], layer_name="Stations")

In [ ]:
m